## **[ Produced by ]**

[Eunchan Lee of KNU-BrainAI](https://github.com/purang2)


**[ 목차 ]**


---



### Requirements 설치 

In [1]:
!pip install transformers 
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.8 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
     |████████████████████████████████| 596 kB 82.5 MB/s 
     |████████████████████████████████| 6.6 MB 56.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 117 kB 9.5 MB/s 


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

## Model Loading

Huggingface/transformers 패키지를 통해 토크나이저와 모델을 load 

Model size: 11.5 GB (큰 것 같다 -> 16GB 권장, 고용량의 RAM 필요)
- KoGPT6B-ryan1.5b는 32GB GPU RAM을 최소로 권장 
- KoGPT6B-ryan1.5b-float16는 16GB GPU RAM을 권장



In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16', # or float32 version: revision=KoGPT6B-ryan1.5b
    bos_token = '[BOS]', eos_token='[EOS]', unk_token = '[UNK]', pad_token='[PAD]', mask_token = '[MASK]'
)

model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',# or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

_  = model.eval()

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5G [00:00<?, ?B/s]

Zero-finetune 상태에서도 아래와 같이 잘 생성해낸다. 

In [4]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 수수께끼를 풀 수 있을까. 영화 <루시>는 인류가 아직 풀지 못한, 아니 영원히 풀지 못할 인류의 최대 난제를 건드린다. 바로 뇌를 100% 활용한다는 것


## Examples 

GPT-3는 Train이 없거나 적은 양으로도 태스크를 수행할 수 있는 Few-shot, Zero-shot 환경에서도 매우 우수한 성능을 보여주었다.  

이 예제에서는 GPT-3를 모티브로 개발된 **Kakaobrain/KoGPT**를 통해 한국어 기반 사설 자연어처리 데이터셋을 통해 Few-shot 성능을 검증해본다.

## 3행시 하는 AI 만들기

훈련을 통해 weight를 업데이트하지 않는 Fewshot Learning 개념과 KoGPT 모델을 활용하여 

3행시(사실상 N행시) AI를 만들어 보는 예제이다. 


대충만든 프롬프트로 어떠한 성능을 내는지 지켜보자 !



In [45]:
#3행시를 위한 Prompt 대략적 생성 

#N을 10개 정도로 한정 => 출처: 구글링 
prompt = '한글 단어로 삼행시 짓기:\n'

prompt += '''
FTA =>
F : Fine
T : Thank you
A : And you?

엄마아빠 =>
엄 : 엄마는
마 : 마덜
아 : 아빠는
빠 : 빠덜

소나기 =>
소 : 소방차가 불난 집 불을 끈다.
나 : 나는 신나게 구경을 했다.
기 : 기절했다. 우리 집이었다.

아이돌 =>
아 : 아저씨
이 : 이제 좀 자리로
돌 : 돌아가세요. 안 보여요.

아이유 =>
아 : 아이유
이 : 이휘재
유 : 유재석

엄정화 => 
엄 : 엄청 예쁘구나
정 : 정말 예뻐
화 : 화장발이구나

성탄절 =>
성 : 성탄절에도 너의 애인은
탄 : 탄생하지 않아
절 : 절대로

화재 =>
화 : 화내지 말고 들어요
재 : 재수 없어

펭귄 =>
펭 : 펭현숙
귄 : 귄카 

칠레 =>
칠 : 칠전팔기의 명장
레 : 레면집 사장님 

박명수 =>
박 : 박식하기 그지없고 
명 : 명석한 두뇌를 가진 
수 : 수재요!

아나바다 =>
아 : 아버지
나 : 나를 낳으시고 
바 : 바지 적삼
다 : 다 적시셨네 
'''

prompt

'한글 단어로 삼행시 짓기:\n\nFTA =>\nF : Fine\nT : Thank you\nA : And you?\n\n엄마아빠 =>\n엄 : 엄마는\n마 : 마덜\n아 : 아빠는\n빠 : 빠덜\n\n소나기 =>\n소 : 소방차가 불난 집 불을 끈다.\n나 : 나는 신나게 구경을 했다.\n기 : 기절했다. 우리 집이었다.\n\n아이돌 =>\n아 : 아저씨\n이 : 이제 좀 자리로\n돌 : 돌아가세요. 안 보여요.\n\n아이유 =>\n아 : 아이유\n이 : 이휘재\n유 : 유재석\n\n엄정화 => \n엄 : 엄청 예쁘구나\n정 : 정말 예뻐\n화 : 화장발이구나\n\n성탄절 =>\n성 : 성탄절에도 너의 애인은\n탄 : 탄생하지 않아\n절 : 절대로\n\n화재 =>\n화 : 화내지 말고 들어요\n재 : 재수 없어\n\n펭귄 =>\n펭 : 펭현숙\n귄 : 귄카 \n\n칠레 =>\n칠 : 칠전팔기의 명장\n레 : 레면집 사장님 \n\n박명수 =>\n박 : 박식하기 그지없고 \n명 : 명석한 두뇌를 가진 \n수 : 수재요!\n\n아나바다 =>\n아 : 아버지\n나 : 나를 낳으시고 \n바 : 바지 적삼\n다 : 다 적시셨네 \n\n'

In [62]:
prompts = prompt + ' 깃허브 => \n깃:'


with torch.no_grad():
    tokens = tokenizer.encode(prompts, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=400)
    generated = tokenizer.batch_decode(gen_tokens)[0]



generated[len(prompt):] 

'한글 단어로 삼행시 짓기:\n\nFTA =>\nF : Fine\nT : Thank you\nA : And you?\n\n엄마아빠 =>\n엄 : 엄마는\n마 : 마덜\n아 : 아빠는\n빠 : 빠덜\n\n소나기 =>\n소 : 소방차가 불난 집 불을 끈다.\n나 : 나는 신나게 구경을 했다.\n기 : 기절했다. 우리 집이었다.\n\n아이돌 =>\n아 : 아저씨\n이 : 이제 좀 자리로\n돌 : 돌아가세요. 안 보여요.\n\n아이유 =>\n아 : 아이유\n이 : 이휘재\n유 : 유재석\n\n엄정화 => \n엄 : 엄청 예쁘구나\n정 : 정말 예뻐\n화 : 화장발이구나\n\n성탄절 =>\n성 : 성탄절에도 너의 애인은\n탄 : 탄생하지 않아\n절 : 절대로\n\n화재 =>\n화 : 화내지'

### KOGPT-N행시 Model Demo

In [69]:


while True:
  word = input('입력 단어를 여기에 입력하시오!😁: ')
  
  try:
    prompts = prompt + f'{word} => \n{word[0]}:'
  except: 
    print('입력이 잘못 되었다!')
    continue
  
  with torch.no_grad():
      tokens = tokenizer.encode(prompts, return_tensors='pt').to(device='cuda', non_blocking=True)
      gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=375)
      generated = tokenizer.batch_decode(gen_tokens)[0]

  print("OUTPUT: ", generated[len(prompt):] )

입력 단어를 여기에 입력하시오!😁: GPT
OUTPUT:  GPT => 
G: Good
T: Top.
A : Assh

박지성 =>
박 : 박지성은 
지 : 진지했어요
성 : 성시경이나 되겠지 

홍진영 =>
홍 : 홍진영은 
진 : 진짜예요
영 : 영원불멸의 가수 


입력 단어를 여기에 입력하시오!😁: 성시경
OUTPUT:  성시경 => 
성: 성시경을 잘 모르지만 
시 : 시금치는 몰라도 
경 : 경시대회는 잘해요

설경구 =>
설 : 설경구를 잘 모르지만 
경구 : 경식아!

이순재 =>
이 : 이름이?
순 : 순재요! 
입력 단어를 여기에 입력하시오!😁: 손흥민
OUTPUT:  손흥민 => 
손: 손에 땀이 난다.
웅 : 웅성거리는 사람들,
신: 신기한 듯 모두 나를 쳐다본다.
민 : 민중의 지팡이여

우리말을 사랑합시다.




















입력 단어를 여기에 입력하시오!😁: 네이버
OUTPUT:  네이버 => 
네: 네 글자를 
버: 버스로
이: 이렇게 예쁘게 
아: 아침에 일어나서 
배: 배타고 
아: 아침에 일어나서 
바: 바지 적삼 
다: 다 젖었네 

엄마아빠 =>
엄 : 엄처어
입력 단어를 여기에 입력하시오!😁: 카카오
OUTPUT:  카카오 => 
카: 카레이서
이: 이제 이카레이서가 되었네.

엄마아빠 =>
엄: 엄친아를 둔 친구가 있는데 
마: 마덜한 엄마가 있는데
아: 아빤가?

엄정화 =
엄 : 엄친딸을 둔 친구
입력 단어를 여기에 입력하시오!😁: ㅇ
OUTPUT:  ㅇ => 
ㅇ: 러버덕을 보러가자 
스: 스누피의 하루 
더: 더우면 물속에 풍덩 

영어단어를 이용한 삼행시 지으려면

(1) 단어를 선택한다.

(2) 단어를 이어 문장을 만든다.

(


KeyboardInterrupt: ignored

## **[ References ]**


01. [[paper review] GPT-3 : Language Models are Few-Shot Learners](https://supkoon.tistory.com/27) -> Few-shot Learning, Zero-shot 관련 개념 블로그

02. [Github: Kakaobrain KoGPT](https://github.com/kakaobrain/kogpt#python) -> 코드 형식 및 예제 등 관련

03. [NSMC Datasets: Huggingface](https://huggingface.co/datasets/nsmc) -> 허깅페이스 datasets의 nsmc 문서 관련

---